In [27]:
#TO DO
# присвоить id в таблице кладовщика к каждому ТС и по нему уже делать слияние
# в БД сделать 1 столбец, ибо id в pandas присваивается автоматически 
# БД можно вести в csv-файле
# можно обязать кладовщиков использовать id при добавлении ТС в свои таблицы
# БД готово в виде csv-файла

In [ ]:
import pandas as pd
path = {'bigdata' : "C:\\Users\\Dmitrii\\Desktop\\pandas\\БД ТС Omnicomm.xlsx",
       'report' : "C:\\Users\\Dmitrii\\Downloads\\Сводный отчет.xlsx",
       'uchastok10' : "C:\\Users\\Dmitrii\\Downloads\\Сводный табель Апрель 2021.xlsx",
       'uchastok41' : 'C:\\Users\\Dmitrii\\Downloads\\ГСМ Участок 41 (1).xlsx'}

n = 4 # число месяца

# обработка сводного отчета
df = pd.read_excel(path['report'])
df = df.drop([0, 1, 2, 3, 4, 5, 6])
df.columns = ['name', 'date', 'val']

df1 = df[0:len(df):3]
df0 = df[df.date == 'Итого:']
df1.index = df0.index

result_report = pd.concat([df1[['name', 'date']], df0.val], axis = 'columns')
result_report = result_report[(result_report.val != 0) & (result_report.val != '-')]

# обработка БД
data_bd = pd.read_excel(path['bigdata']) # открываем БД
data_bd = data_bd[data_bd.klad.notnull()] # отфильтровываем избыточные значения с None

In [28]:
def klad(y, z):
    c = 'Unnamed: 3'
    if z == 1:
        c = 'Unnamed: 3'
    else:
        c = list(c)
        c[-1] = str(int(c[-1]) + (z - 1))
        c = ''.join(c)
    x = pd.read_excel(y) # откываем таблицу кладовщика
    x.drop(x.columns.difference(['Unnamed: 1', c]), 1, inplace=True) # берем 2 колонки: название ТС и заправку
    x.columns = ['klad', 'g'] # переименоввываем столбцы
    return x

In [29]:
uchastok10 = klad(path['uchastok10'], n)

In [30]:
uchastok41 = klad(path['uchastok41'], n)

In [31]:
# объединяем таблицы
df = pd.concat([uchastok10, uchastok41])
df1 = pd.merge(data_bd, df)
df1 = df1[df1.g.notnull()]

df1 = df1.dropna(axis='index', how='any', subset=['g', 'klad']).drop('klad', axis=1)

In [32]:
result = pd.merge(result_report, df1, left_on = 'name', right_on = 'Omnicomm').drop('Omnicomm', axis=1) #.drop('Unnamed: 4', axis=1)

In [33]:
result['-'] = result['g'] - result['val'] # формула расхождения литража

In [26]:
result

,name,date,val,g,-
0,Denyo DCW [109523],04.04.2021,45.5,72,26.5
1,Shindaiwa [111341],04.04.2021,39.3,38,-1.3
2,Shindaiwa [112154],04.04.2021,56.8,68,11.2
3,W.Neuson LTN8L [112455],04.04.2021,19.7,30,10.3
4,W.Neuson LTN8L [112456],04.04.2021,55.5,58,2.5
5,Автобус Газ-3295А1 [а 910 ун 763],04.04.2021,70,73,3
6,Автобус УРАЛ-3255-5013-71 [в 641 мт 89],04.04.2021,106.8,105,-1.8
7,Автокран УРАЛ-КС 55733 [а 928 рв 763],04.04.2021,161.5,165,3.5
8,АД-100 [109238],04.04.2021,215.9,231,15.1
9,Водовоз УРАЛ-4320-УСТ 45322 [В 850 МТ 89],04.04.2021,184.8,197,12.2


In [34]:
result.to_excel("result.xlsx")